In [1]:
import pickle
import sqlite3
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

model = model.cuda()

In [ ]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/bcu_caption.db")
Photo = pd.read_sql_query("SELECT * from Photo", con)

# Verify that result of SQL query is stored in the dataframe
Photo.keys()

In [ ]:
def translate(sentence):
    input_ids = tokenizer(sentence, return_tensors="pt").input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, num_beams=5, num_return_sequences=1)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [ ]:
Photo['title_en'] = Photo['title'].apply(lambda x: translate(x) if pd.notna(x) else '')

In [ ]:
Photo.to_sql("Photo",con,if_exists='replace',index=False, index_label="id")